# 4. Expressions for Optimization Models

This notebook is part of our step-by-step guide to creating optimization models. We continue our 7-step procedure:

1. ✓ Import Pyomo and Define the Sets
2. ✓ Define the Decision Variables
3. ✓ Define the Parameters
4. ➡️ Define the Expressions
5. Define the Objective Function
6. Define the Constraints

In this notebook, we explore how to create and manage expressions in Pyomo. Expressions combine variables and parameters using mathematical operations. They form the building blocks for objective functions and constraints.

We will cover:
- What are expressions and why they matter
- How to create basic expressions
- How to store expressions in the model
- Common expression patterns
- Best practices for managing expressions

## Setup: Creating Our Model Environment

Let's set up our model with some basic components from previous notebooks:

In [1]:
import pyomo.environ as pyo

# Create model
m = pyo.ConcreteModel()

# Define sets
m.I = pyo.Set(initialize=['P1', 'P2', 'P3'])  # Products
m.J = pyo.Set(initialize=['M1', 'M2'])        # Machines

# Define variables
m.x = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize={i: 100 for i in m.I}) # production of product i
m.y = pyo.Var(m.I, m.J, domain=pyo.Binary) # production of product i on machine j

# Define parameters
m.p = pyo.Param(m.I, initialize={'P1': 10, 'P2': 15, 'P3': 20})  # Prices
m.c = pyo.Param(m.J, initialize={'M1': 50, 'M2': 40})           # Machine costs
m.d = pyo.Param(m.I, initialize={'P1': 80, 'P2': 120, 'P3': 60}) # Demand

## Creating and Storing Expressions

In Pyomo, we can create expressions in two ways:
1. As temporary expressions that we use immediately
2. As stored expressions that become part of our model

Let's look at both approaches:

In [2]:
# Temporary expression - revenue calculation
revenue = sum(m.p[i] * m.x[i] for i in m.I)
print(f"Temporary expression: {revenue}")
print(f"Expression value: {revenue()}")

Temporary expression: 10*x[P1] + 15*x[P2] + 20*x[P3]
Expression value: 4500


In [3]:
# Stored expression - becomes part of the model
m.revenue = pyo.Expression(expr=sum(m.p[i] * m.x[i] for i in m.I))
m.machine_usage = pyo.Expression(m.J, rule=lambda m, j: sum(m.y[i,j] for i in m.I))

The key difference is that stored expressions:
- Are accessible throughout the model
- Can be referenced in other expressions
- Can be monitored and analyzed after solving
- Make the model more readable and maintainable

## Common Expression Patterns

Let's explore some typical expression patterns in optimization models:

In [4]:
# 1. Simple summation
m.total_production = pyo.Expression(expr=sum(m.x[i] for i in m.I))

# 2. Weighted sum
m.total_cost = pyo.Expression(
    expr=sum(m.c[j] * sum(m.y[i,j] for i in m.I) for j in m.J)
)

# 3. Ratio expression
m.avg_price = pyo.Expression(
    expr=sum(m.p[i] * m.x[i] for i in m.I) / sum(m.x[i] for i in m.I)
)

# 4. Conditional summation
def capacity_rule(m):
    return sum(m.y[i,j] for i in m.I for j in m.J 
              if m.d[i] > 100)  # Only for high-demand products
m.high_demand_capacity = pyo.Expression(rule=capacity_rule)

## Using Expressions in Constraints and Objectives

Stored expressions make our constraints and objectives much clearer:

In [5]:
# Define profit expression
m.profit = pyo.Expression(expr=m.revenue - m.total_cost)

# Use in objective (we'll cover this more in the next notebook)
m.obj = pyo.Objective(expr=m.profit, sense=pyo.maximize)

# Use in constraints (we'll cover this more later)
m.min_profit = pyo.Constraint(expr=m.profit >= 1000)
m.capacity = pyo.Constraint(m.J,
    rule=lambda m, j: sum(m.x[i] * m.y[i,j] for i in m.I) <= 500)

## Best Practices for Expressions

When working with expressions in Pyomo:

1. Use meaningful names that reflect what the expression represents

2. Store expressions that will be used multiple times

3. Break complex expressions into smaller, more manageable parts

4. Use expression rules for complex logic

5. Document the meaning and units of expressions

Here's an example applying these practices:

In [6]:
# Clear expression names
def production_cost_rule(m, i):
    """Calculate total production cost for product i across all machines"""
    return sum(m.c[j] * m.y[i,j] for j in m.J)

def revenue_rule(m, i):
    """Calculate revenue for product i based on price and production"""
    return m.p[i] * m.x[i]

# Store expressions with documentation
m.production_cost = pyo.Expression(m.I, rule=production_cost_rule)
m.product_revenue = pyo.Expression(m.I, rule=revenue_rule)
m.product_profit = pyo.Expression(m.I, 
    rule=lambda m, i: m.product_revenue[i] - m.production_cost[i])

## Practice Exercises

1. Use the model setup from the beginning of the notebook to create the model, sets, variables and parameters.
2. Create the expression for total production $$Q = \sum_{i \in I} x_i $$
3. Create the expression for utilization percentage on each machine $$u_i = \frac{x_i}{Q}$$
4. Create the expression for total profit $$\text{profit} = \sum_{i \in I} \text{profit of product i} \times x_i $$ 
5. Create the expression for total demand satisfaction percentage $$ds_i = \frac{x_i}{\text{demand of product i}} $$
6. Create the expression for machine utilization balance (max vs min usage) $$b = \frac{\text{max utilization percentage}}{\text{min utilization percentage}}$$

In [7]:
#1. Set up the model (copy from the beginning of the notebook)


In [8]:
#2. Create the expression for total production $$Q = \sum_{i \in I} x_i $$


In [9]:
#3. Create the expression for utilization percentage on each machine $$u_i = x_i/Q$$


In [10]:
#4. Create the expression for total profit $$\sum_{i \in I} profit of product i * x_i $$ 


In [11]:
#5. Create the expression for total demand satisfaction percentage $$ds_i = x_i/demand of product i$$


In [12]:
#6. Create the expression for machine utilization balance (max vs min usage) $$b = max(u_i)/min(u_i)$$